In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=eafe98ed4e7fd3afe0cb418b31a5b3151cd3eda5f8f1966f5c15d08656ae2238
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import requests
import zipfile
from pyspark.sql import SparkSession

In [3]:
url = 'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip'

In [4]:
r = requests.get(url, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
with open("r.zip", "wb") as f:
    f.write(r.content)

In [6]:
with zipfile.ZipFile("r.zip", "r") as zip_ref:
    zip_ref.extractall("./microData")

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
df = spark.read.csv("microData/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", sep=';', header=True, inferSchema=True)

In [9]:
from pyspark.sql.functions import col

df = df.select([col(c).alias(c.lower()) for c in df.columns])

In [10]:
df.write.parquet("MICRODADOS_IES_CONSULTA.parquet")

parquet_df = spark.read.parquet("MICRODADOS_IES_CONSULTA.parquet")

parquet_df.createOrReplaceTempView("df_parquet_vitor")

####  Olhando para o estado de Minas Gerais, quantos municípios têm informações presentes na base de dados?


In [22]:
df1 = spark.sql("""
    SELECT COUNT(DISTINCT no_municipio_ies) 
    FROM df_parquet_vitor
    WHERE sg_uf_ies = 'MG'
""")
df1.show()

+--------------------------------+
|count(DISTINCT no_municipio_ies)|
+--------------------------------+
|                             104|
+--------------------------------+



#### Quantos professores doutores existem em cada cidade de Minas Gerais presente na base de dados 

In [23]:
df2 = spark.sql("""
    SELECT no_municipio_ies nome_cidade, SUM(qt_doc_ex_dout) numero_doutores_por_cidade  
    FROM df_parquet_vitor
    WHERE sg_uf_ies = 'MG'
    GROUP BY no_municipio_ies
    ORDER BY numero_doutores_por_cidade DESC
""")

df2.show()

+--------------------+--------------------------+
|         nome_cidade|numero_doutores_por_cidade|
+--------------------+--------------------------+
|      Belo Horizonte|                      6615|
|          Uberl�ndia|                      1939|
|        Juiz de Fora|                      1915|
|              Vi�osa|                      1185|
|       Montes Claros|                       925|
|             Uberaba|                       891|
|              Lavras|                       812|
|    S�o Jo�o del Rei|                       753|
|          Ouro Preto|                       746|
|          Diamantina|                       673|
|             Alfenas|                       661|
|        Pouso Alegre|                       499|
|             Itajub�|                       304|
|            Contagem|                       219|
|         Sete Lagoas|                       201|
|               Betim|                       192|
|         Divin�polis|                       133|


####  Qual a quantidade de docentes com deficiência no estado do Paraná?


In [19]:
df3 = spark.sql("""
    SELECT SUM(qt_doc_ex_com_deficiencia) AS pessoas_deficientes_parana 
    FROM df_parquet_vitor
    WHERE sg_uf_ies = 'PR'
""")

df3.show()

+--------------------------+
|pessoas_deficientes_parana|
+--------------------------+
|                       160|
+--------------------------+



In [ ]:
df1.write.parquet('questao1')
df2.write.parquet('questao2')
df3.write.parquet('questao3')